In [1]:
import pandas as pd
import numpy as np
import os
from numpy import random
import scipy 
from scipy import stats
from matplotlib import pyplot as plt
from scipy.stats import binom
from scipy.stats import poisson
from scipy.stats import norm ,t
import seaborn as sns
import pylab
from pylab import legend,plot,show,title,xlabel,ylabel
from random import choice

import statsmodels
from statsmodels import stats
from statsmodels.stats import weightstats as ssw
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats import proportion  as ssp

from scipy.stats import chisquare
from scipy.stats import chi2_contingency

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from statsmodels.stats.outliers_influence import variance_inflation_factor

from statsmodels import discrete
from statsmodels.discrete import discrete_model
from statsmodels.discrete.discrete_model import MNLogit
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.feature_selection import SelectKBest

## poisson regression
import statsmodels.formula.api as smf
from statsmodels.discrete.discrete_model import Poisson as psn

from statsmodels.discrete.discrete_model import NegativeBinomial as ngb

In [2]:
df=pd.read_csv("day.csv")

In [3]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,1,1,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,2,1,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,3,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,4,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,5,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [4]:
df.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    int64  
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.4+ KB


In [6]:
categorical_data=["season","yr","holiday","workingday","weathersit"]
data_cleaned=df.drop(columns=["instant","dteday","casual","registered"])
data_cleaned=pd.get_dummies(data_cleaned,columns=categorical_data,drop_first=True).astype(int)

In [7]:
data_cleaned

,mnth,weekday,temp,atemp,hum,windspeed,cnt,season_2,season_3,season_4,yr_1,holiday_1,workingday_1,weathersit_2,weathersit_3
0,1,1,14,18,80,10,985,0,0,0,0,0,1,1,0
1,1,2,14,17,69,16,801,0,0,0,0,0,1,1,0
2,1,3,8,9,43,16,1349,0,0,0,0,0,1,0,0
3,1,4,8,10,59,10,1562,0,0,0,0,0,1,0,0
4,1,5,9,11,43,12,1600,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,12,5,10,11,65,23,2114,0,0,0,1,0,1,1,0
726,12,6,10,12,59,10,3095,0,0,0,1,0,0,1,0
727,12,0,10,12,75,8,1341,0,0,0,1,0,0,1,0
728,12,1,10,11,48,23,1796,0,0,0,1,0,1,0,0


In [8]:
# x=df.drop("cnt",axis=1)
# y=df.cnt
# res=SelectKBest(k=4).fit(x,y)
# d1={'Factors':x.columns,'Scores':res.scores_}
# NewRes=pd.DataFrame(d1)   # used to tell the factors affecting response

In [9]:
features=data_cleaned[["temp","atemp","hum","windspeed"]]
response=data_cleaned.cnt
x_train,x_test,y_train,y_test=train_test_split(features,response,test_size=0.2,random_state=20)  # order of variable should be same
x_train=sm.add_constant(x_train,prepend=False)
model=sm.OLS(y_train,x_train).fit()

In [10]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    cnt   R-squared:                       0.473
Model:                            OLS   Adj. R-squared:                  0.469
Method:                 Least Squares   F-statistic:                     129.8
Date:                Wed, 25 Dec 2024   Prob (F-statistic):           4.39e-79
Time:                        14:15:55   Log-Likelihood:                -5062.6
No. Observations:                 584   AIC:                         1.014e+04
Df Residuals:                     579   BIC:                         1.016e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
temp        -114.1526     84.305     -1.354      0.1

In [11]:
## here we do not reject null hypothesis of temp since pvalue>0.05 means tenmp does not affect cnt
##so we are removing temp

In [12]:
x_test=sm.add_constant(x_test,prepend=False)
y_pred=model.predict(x_test)

In [13]:
from sklearn.metrics import r2_score
print("R2 score:",r2_score(y_test,y_pred))   #  this value telling that this model working more worse on testing data

R2 score: 0.3702847491449419


In [14]:
### now checking on other factors

In [15]:
features=data_cleaned[["atemp","hum","windspeed","season_2","season_3","season_4"]]
response=data_cleaned.cnt
x_train,x_test,y_train,y_test=train_test_split(features,response,test_size=0.2,random_state=20)  # order of variable should be same
x_train=sm.add_constant(x_train,prepend=False)
model=sm.OLS(y_train,x_train).fit()

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    cnt   R-squared:                       0.535
Model:                            OLS   Adj. R-squared:                  0.530
Method:                 Least Squares   F-statistic:                     110.6
Date:                Wed, 25 Dec 2024   Prob (F-statistic):           1.41e-92
Time:                        14:15:55   Log-Likelihood:                -5025.9
No. Observations:                 584   AIC:                         1.007e+04
Df Residuals:                     577   BIC:                         1.010e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
atemp        133.6550     11.824     11.304      0.0

In [20]:
for i in range(data_cleaned.shape[1]):
    print("column",i,"-",data_cleaned.columns[i],"has VIF factor as",variance_inflation_factor(data_cleaned.values,i))

column 0 - mnth has VIF factor as 15.356830130672515
column 1 - weekday has VIF factor as 3.167308707956507
column 2 - temp has VIF factor as 476.3150428499145
column 3 - atemp has VIF factor as 506.19424117819165
column 4 - hum has VIF factor as 22.758965290741607
column 5 - windspeed has VIF factor as 5.134514868387152
column 6 - cnt has VIF factor as 0.7511104957055933
column 7 - season_2 has VIF factor as 4.036118724553486
column 8 - season_3 has VIF factor as 7.775988616776718
column 9 - season_4 has VIF factor as 7.0787235249593365
column 10 - yr_1 has VIF factor as 5.129649051795746
column 11 - holiday_1 has VIF factor as 1.1302739969200783
column 12 - workingday_1 has VIF factor as 3.4666553911028752
column 13 - weathersit_2 has VIF factor as 2.4301369554225367
column 14 - weathersit_3 has VIF factor as 1.4612699512421157
